In [1]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [2]:
# data = pd.read_table("question1.txt")
# data.head()
# data.to_csv('question1.csv', encoding='utf-8', index=False)

In [112]:
data = pd.read_csv("question2.csv")
# data['weight']= data['weight']+1
data.head(15)

,user_id,weight,response
0,1,2,Make sure to try it in the store- go stand aga...
1,2,3,Consider what kind of sleeper you are and how ...
2,3,0,Personally went to a mattress store and they h...
3,4,2,"Former relax the back employee here, imagine o..."
4,5,1,Pacific Coast pillows are incredibly comfortab...
5,6,2,"If you lay down on your side, your pillow (and..."
6,7,2,Most people will say the store but I shopped a...
7,8,0,I have a MyPillow and a weighted blanket. I sl...
8,9,0,Chiropractors and physiotherapists usually sel...
9,10,3,I've found my best ones at places like TJ max....


In [113]:
# data['response'][8] = 'lay on your side, outer leg straight and inner leg crooked with your foot under your knee. inner arm bent with hand under head on a kinda firm pillow, fingers near the back of your head, outer hand resting on your elbow, pillow lying at your back and slightly under near your waist.\r\n\r\n I stole this from the film Equilibrium, everythings clinical and I assume the looked into the best way to sleep. tried it out and it\'s surprisingly comfortable once you balance it out with a pillow at your back .'

In [114]:
# data['response'][0] 

In [115]:
responses = []
data['response']=data['response'].values.astype('U')
for i in range(0, len(data)):
    text = data['response'][i]
    #strips html formatting and converts to unicode
    responses.append(text)

# responses = synopses_clean_imdb

print(str(len(responses)) + ' responses.')

15 responses.


In [116]:
# synopses_imdb[0][100]

In [117]:
# synopses = []

# for i in range(len(synopses_imdb)):
#     item = synopses_imdb[i]
#     synopses.append(item)

In [119]:
# load nltk's English stopwords as variable called 'stopwords'
stopwords = nltk.corpus.stopwords.words('english')

In [120]:
# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [121]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [122]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in responses:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [123]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [124]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.99, max_features=20000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(responses)

print(tfidf_matrix.shape)

Wall time: 67.5 ms
(15, 1309)


In [128]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

# K-means clustering

In [162]:
from sklearn.cluster import KMeans

num_clusters = 5
km = KMeans(n_clusters=num_clusters, random_state=123)

%time km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

Wall time: 109 ms


In [163]:
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=123, tol=0.0001, verbose=0)

In [165]:
films = {'response': responses, 'user_id': data['user_id'].values, 'weight': data['weight'].values, 'cluster_no': clusters}

frame = pd.DataFrame(films, index = [clusters] , columns = ['cluster_no', 'user_id', 'weight', 'response'])

In [166]:
# data['user_id'].values

In [167]:
frame.head(15)

,cluster_no,user_id,weight,response
2,2,1,2,Make sure to try it in the store- go stand aga...
0,0,2,3,Consider what kind of sleeper you are and how ...
4,4,3,0,Personally went to a mattress store and they h...
0,0,4,2,"Former relax the back employee here, imagine o..."
2,2,5,1,Pacific Coast pillows are incredibly comfortab...
3,3,6,2,"If you lay down on your side, your pillow (and..."
1,1,7,2,Most people will say the store but I shopped a...
1,1,8,0,I have a MyPillow and a weighted blanket. I sl...
1,1,9,0,Chiropractors and physiotherapists usually sel...
1,1,10,3,I've found my best ones at places like TJ max....


In [168]:
clus_frame = frame.reset_index()

In [169]:

clus_frame = clus_frame.drop(['level_0'], axis=1)
clus_frame

,cluster_no,user_id,weight,response
0,2,1,2,Make sure to try it in the store- go stand aga...
1,0,2,3,Consider what kind of sleeper you are and how ...
2,4,3,0,Personally went to a mattress store and they h...
3,0,4,2,"Former relax the back employee here, imagine o..."
4,2,5,1,Pacific Coast pillows are incredibly comfortab...
5,3,6,2,"If you lay down on your side, your pillow (and..."
6,1,7,2,Most people will say the store but I shopped a...
7,1,8,0,I have a MyPillow and a weighted blanket. I sl...
8,1,9,0,Chiropractors and physiotherapists usually sel...
9,1,10,3,I've found my best ones at places like TJ max....


In [174]:
print(frame.groupby(['cluster_no']).count())
print(frame.groupby(['cluster_no']).mean())

            user_id  weight  response
cluster_no                           
0                 4       4         4
1                 5       5         5
2                 2       2         2
3                 2       2         2
4                 2       2         2
            user_id  weight
cluster_no                 
0              8.25     1.5
1              9.60     1.0
2              3.00     1.5
3              8.50     2.0
4              8.00     0.0


In [175]:
summary1 = frame.groupby(['cluster_no']).count()
summary1 = summary1.drop(['user_id', 'weight'], axis=1)

In [176]:
summary2 = frame.groupby(['cluster_no']).mean()
summary2 = summary2.drop(['user_id'], axis=1)

In [190]:
summary = summary1.join(summary2)
summary['avg_weight'] = summary['weight']
summary['truth_score'] = summary['response'] * summary['avg_weight']
summary['reponse_count'] = summary['response']
summary = summary.drop(['weight', 'response'], axis=1)
summary = summary[['reponse_count', 'avg_weight', 'truth_score']]
summary

,reponse_count,avg_weight,truth_score
cluster_no,,,
0,4,1.5,6.0
1,5,1.0,5.0
2,2,1.5,3.0
3,2,2.0,4.0
4,2,0.0,0.0


In [178]:
from __future__ import print_function

print("Top terms per cluster:")
print()
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
response_text = []
text_str = ""
for i in range(num_clusters):
    print('\n____________________________________________________________________________________')
    print("Cluster %d words:" % i, end='')
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0], end=',')
    print()
    print()
    print("Cluster %d Responses:" % i, end='')
    print('\n____________________________')
    response_text.append(" ")
    for title in frame.ix[i]['response'].values.tolist():
        print()
        response_text[i] = text_str = " "+ response_text[i] + " " + title
        print(' %s,' % title, end='')
        print()
        print()

Top terms per cluster:


____________________________________________________________________________________
Cluster 0 words: pillow, need, head, sleep, n't, firmer,

Cluster 0 Responses:
____________________________

 Consider what kind of sleeper you are and how heavy your head is. If you sleep on your back you need a soft pillow that won't prop your head up too much and you want to position it near your neck to give you support, if your a side sleeper you need a firmer pillow that will hold your head level to your body. If your head is heavy, go firmer, if your head is light like mine you need very soft pillows so you don't end up with a crick in your neck. If you tend to change positions in your sleep find the one you think will work best but have two pillows, one a little better for side sleeping and another for back sleeping and keep them on the bed, at night you'll hopefully use them both.

Try them in store and get weird with it if you have to, put your head on that stuff. Als

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [179]:
from gensim.summarization.summarizer import summarize
from gensim.summarization import keywords
 
import requests
     
#getting text document from web, below function based from 3
from bs4 import BeautifulSoup
from urllib.request import urlopen
 
text = ('Sleep',
 response_text[0])
 
print ('Summary:')   
print (summarize(str(text), ratio=0.3))
 
print ('\nKeywords:')
 
# higher ratio => more keywords
print (keywords(str(text), ratio=0.2))

Summary:
If your head is heavy, go firmer, if your head is light like mine you need very soft pillows so you don't end up with a crick in your neck.
If you tend to change positions in your sleep find the one you think will work best but have two pillows, one a little better for side sleeping and another for back sleeping and keep them on the bed, at night you'll hopefully use them both.\n\nTry them in store and get weird with it if you have to, put your head on that stuff.
If your testing the pillow on a hard surface but your bed is soft than that you will probably have to go firmer than what you find immediately comfortable.
If you don't want too buy like five different pillows just pick the one you think is comfortable and depending on the firmness of your bed go firmer or softer.
If you sleep on your side you need a soft pillow that won't prop your head up too much and you want to position it near your neck to give you support, if your a side sleeper you need a firmer pillow that wi

In [262]:
# text

In [180]:
clus_frame.head(15)

,cluster_no,user_id,weight,response
0,2,1,2,Make sure to try it in the store- go stand aga...
1,0,2,3,Consider what kind of sleeper you are and how ...
2,4,3,0,Personally went to a mattress store and they h...
3,0,4,2,"Former relax the back employee here, imagine o..."
4,2,5,1,Pacific Coast pillows are incredibly comfortab...
5,3,6,2,"If you lay down on your side, your pillow (and..."
6,1,7,2,Most people will say the store but I shopped a...
7,1,8,0,I have a MyPillow and a weighted blanket. I sl...
8,1,9,0,Chiropractors and physiotherapists usually sel...
9,1,10,3,I've found my best ones at places like TJ max....


In [181]:
best_cluster = 0

In [182]:
for i in range(0,len(clus_frame)):
    if clus_frame['cluster_no'][i] == best_cluster:
        if clus_frame['weight'][i]<3:
            print(clus_frame['weight'][i])
            clus_frame['weight'][i] = clus_frame['weight'][i]+1
    else:
        if clus_frame['weight'][i]>0:
            clus_frame['weight'][i] = clus_frame['weight'][i]-1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


2
1
0


In [189]:
clus_frame['prev_weight'] = frame['weight'].values
clus_frame.head(15)

,cluster_no,user_id,weight,response,prev_weight
0,2,1,1,Make sure to try it in the store- go stand aga...,2
1,0,2,3,Consider what kind of sleeper you are and how ...,3
2,4,3,0,Personally went to a mattress store and they h...,0
3,0,4,3,"Former relax the back employee here, imagine o...",2
4,2,5,0,Pacific Coast pillows are incredibly comfortab...,1
5,3,6,1,"If you lay down on your side, your pillow (and...",2
6,1,7,1,Most people will say the store but I shopped a...,2
7,1,8,0,I have a MyPillow and a weighted blanket. I sl...,0
8,1,9,0,Chiropractors and physiotherapists usually sel...,0
9,1,10,2,I've found my best ones at places like TJ max....,3


In [184]:
clus_frame.to_csv('updated_q2.csv', encoding='utf-8', index=False)